#  Fe-Ti oxide geothermobarometer
## Constructed based on Fe-Ti exchange for oxide pairs, using the methodology of Ghiorso and Evans (2008)

Required Python code to load the phase library ...

In [1]:
import numpy as np
from scipy import optimize as optim
import thermoengine as thermo
import pandas as pd #for loading Excel sheets

### Get access to a thermodynamic database (by default, the Berman (1988)/MELTS database).

In [2]:
modelDB = thermo.model.Database()

### Create a Python reference to the Spinel ("Mag") and Rhombehedral oxide ("Ilm") solution phase class.

In [3]:
Mag = modelDB.get_phase('SplS')
Ilm = modelDB.get_phase('Rhom')

### Optional - Obtain some properties of the selected Oxide solution  
Name, formulas of endmembers, molecular weights of endmembers, abbreviation, number of endmember components, names of endmembers

In [4]:
print (Mag.props['phase_name'])
print ('Num of endmembers: ', Mag.props['endmember_num'])
print (Mag.props['formula'])
print (Mag.props['endmember_name'])
print ()
print (Ilm.props['phase_name'])
print ('Num of endmembers: ', Ilm.props['endmember_num'])
print (Ilm.props['formula'])
print (Ilm.props['endmember_name'])

Spinel
Num of endmembers:  5
['FeCr2O4' 'FeAl2O4' 'Fe3O4' 'MgAl2O4' 'Fe2TiO4']
['chromite' 'hercynite' 'magnetite' 'spinel' 'ulvospinel']

Ilmenite ss
Num of endmembers:  5
['MgTiO3' 'Fe2O3' 'FeTiO3' 'MnTiO3' 'Al2O3']
['geikielite' 'hematite' 'ilmenite' 'pyrophanite' 'corundum']


# Equations and functions for Fe-Ti oxide geothermometer.
## Consider Fe-Ti exchange between oxides
### Rhom(Ilm) + Spinel(Mag) = Spinel (Ulv) + Rhom(Hm)
### FeTiO3 + Fe3O4 = Fe2TiO4 + Fe2O3

## Correction terms from Ghiorso and Evans (2008) that modify the MELTS models 
### Correction terms for ulvospinel derived in Ghiorso and Evans (2008)

In [5]:
def UlvCorr(t, correctReaction=True):
    tr = 298.15
    h = - 162.0 + 284.5
    s = 0.0
    if correctReaction:
        h += 2039.106175 
        s +=    1.247790
    l1 = - 0.039452*np.sqrt(4.184)
    l2 = 7.54197e-5*np.sqrt(4.184)
    h = h + 0.5*l1*l1*(t*t-tr*tr) + (2.0/3.0)*l1*l2*(t*t*t - tr*tr*tr) + 0.25*l2*l2*(t*t*t*t - tr*tr*tr*tr)
    s = s + l1*l1*(t - tr) + l1*l2*(t*t - tr*tr) + (1.0/3.0)*l2*l2*(t*t*t - tr*tr*tr)
    return h - t*s

### Ghiorso and Evans (2008) used the Vinet integral; MELTS uses the Berman integral
We must substract the latter from computed chemical potentials and add in the former.

In [6]:
def BermanVint(t, p, v0, v1, v2, v3, v4):
    pr = 1.0
    tr = 298.15
    return v0*((v1/2.0-v2)*(p*p-pr*pr)+v2*(p*p*p-pr*pr*pr)/3.0 + (1.0-v1+v2+v3*(t-tr)+v4*(t-tr)*(t-tr))*(p-pr))
def VinetVint(t, p, v0, alpha, K, Kp):
    eta = 3.0*(Kp-1.0)/2.0
    x   = 1.0
    x0  = 1.0
    pr  = 1.0
    tr  = 298.15
    
    iter = 0
    while True:
        fn = x*x*(p/10000.0) - 3.0*K*(1.0-x)*np.exp(eta*(1.0-x)) - x*x*alpha*K*(t-tr)
        dfn = 2.0*x*(p/10000.0) + 3.0*K*(1.0+eta*(1.0-x))*np.exp(eta*(1.0-x)) - 2.0*alpha*K*(t-tr)
        x = x - fn/dfn
        iter += 1
        if ((iter > 500) or (fn*fn < 1.0e-15)):
            break
    # print (iter, x)
    
    iter = 0
    while True:
        fn = x0*x0*(pr/10000.0) - 3.0*K*(1.0-x0)*np.exp(eta*(1.0-x0)) - x0*x0*alpha*K*(t-tr)
        dfn = 2.0*x0*(pr/10000.0) + 3.0*K*(1.0+eta*(1.0-x0))*np.exp(eta*(1.0-x0)) - 2.0*alpha*K*(t-tr)
        x0 = x0 - fn/dfn
        iter += 1
        if ((iter > 500) or (fn*fn < 1.0e-15)):
            break
    # print (iter, x0)
    
    a  = (9.0*v0*K/(eta*eta))*(1.0 - eta*(1.0-x))*np.exp(eta*(1.0-x))
    a += v0*(t-tr)*K*alpha*(x*x*x - 1.0) - 9.0*v0*K/(eta*eta)
    a -= (9.0*v0*K/(eta*eta))*(1.0 - eta*(1.0-x0))*np.exp(eta*(1.0-x0))
    a -= v0*(t-tr)*K*alpha*(x0*x0*x0 - 1.0) - 9.0*v0*K/(eta*eta)
    
    return -a*10000.0 + p*v0*x*x*x - pr*v0*x0*x0*x0

### Berman integral for the reaction FeTiO3 + Fe3O4 = Fe2TiO4 + Fe2O3

In [7]:
def rBerVint(T, P):
    vIntBerMag = BermanVint(T, P, 4.452, -0.582E-6, 1.751E-12, 30.291E-6, 138.470E-10)
    vIntBerUlv = BermanVint(T, P, 4.682, 0.0, 0.0, 0.0, 0.0)
    vIntBerHem = BermanVint(T, P, 3.027, -0.479e-6, 0.304e-12, 38.310e-6, 1.650e-10)
    vIntBerIlm = BermanVint(T, P, 3.170, -0.584e-6, 1.230e-12, 27.248e-6, 29.968e-10)
    return vIntBerUlv + vIntBerHem - vIntBerMag -vIntBerIlm

### Vinet integral for the reaction FeTiO3 + Fe3O4 = Fe2TiO4 + Fe2O3

In [8]:
def rVinetVint(T, P):
    vIntVinetMag = VinetVint(T, P, 4.452, 30.291E-6, 171.821, 9.3387)
    vIntVinetUlv = VinetVint(T, P, 4.682, 30.291E-6, 171.821, 9.3387)
    vIntVinetHem = VinetVint(T, P, 3.027, 38.310E-6, 208.768, 1.64992)
    vIntVinetIlm = VinetVint(T, P, 3.170, 27.248E-6, 171.233, 6.21289)
    return vIntVinetUlv + vIntVinetHem - vIntVinetMag - vIntVinetIlm

### This method computes the free energy of the exchange reaction ...

In [9]:
def deltaG(T, P, mag_mols, ilm_mols):
    muMag = Mag.chem_potential(T, P, mol=mag_mols, endmember=2)
    muUlv = Mag.chem_potential(T, P, mol=mag_mols, endmember=4) + UlvCorr(T)
    muIlm = Ilm.chem_potential(T, P, mol=ilm_mols, endmember=2)
    muHem = Ilm.chem_potential(T, P, mol=ilm_mols, endmember=1)
    deltaG = muUlv + muHem - muIlm - muMag - rBerVint(T, P) + rVinetVint(T, P)
    return deltaG

### This next function is used by the minimizer to zero the free energy of the exchange reaction ...

In [10]:
def boundary(P, Tlims, deltaG, mag_mols, ilm_mols):
    Afun = lambda T, P=P: deltaG(T, P, mag_mols, ilm_mols)
    Tbound = optim.brentq(Afun, Tlims[0], Tlims[1])
    return Tbound

# Step 1 - Input compositions of oxide pairs
### Updating to include load from Excel file (K. Prissel)
Use "GE08_test.xlsx" for example template.

In [11]:
excel_file = 'GE08_test.xlsx'
data = pd.read_excel(excel_file, header=1) #loads Excel file into data frame, gets headers from row 2

In [12]:
#rows if you want to look at all data (both spinel and rhomb. ox) at once
data.columns = ['Index', 'Label', 'Spinel SiO2', 'Spinel TiO2', 'Spinel Al2O3', 'Spinel V2O3',
                            'Spinel Cr2O3', 'Spinel FeO', 'Spinel MnO', 'Spinel MgO', 'Spinel CaO', 'Spinel ZnO', 'Spinel NiO',
                            'Rhomb SiO2', 'Rhomb TiO2', 'Rhomb Al2O3', 'Rhomb V2O3', 'Rhomb Cr2O3', 'Rhomb FeO', 'Rhomb MnO',
                            'Rhomb MgO', 'Rhomb CaO', 'Rhomb ZnO', 'Rhomb NiO']

#data.head() #look at first 5 rows (or input value for rows you want to inspect)

### Reading the spinel and rhombohedral oxide columns into different variables

In [13]:
spinel = pd.read_excel(excel_file, usecols=12, header=1)
#tells read_excel to read in the first columns until index 12 (first 13 columns)
#could also pass a list of numbers that defines which columns to load
spinel.head()

,Index,Label,SiO2,TiO2,Al2O3,V2O3,Cr2O3,FeO,MnO,MgO,CaO,ZnO,NiO
0,1,Hat Creek (Olivine Basalt) Table 5 (63) No: 12...,0.14,26.1,0.41,0.55,0.02,68.8,0.67,0.87,0.10,0.09,0.0
1,2,G151 Rhyolite (Thingmuli),0.13,21.5,1.15,0.00,0.00,73.6,0.88,0.32,0.03,0.23,0.0
2,3,"Oraefi (Obsidian from Oraefajokull, Eastern Ic...",0.11,22.1,0.64,0.00,0.01,73.0,1.00,0.06,0.04,0.27,0.0
3,4,"AC-5 Pitchstone (sill, Cluachland shore, Arran...",0.07,19.8,0.99,0.30,0.10,74.0,0.76,0.40,0.01,0.22,0.0
4,5,"AC-7 Pitchstone (dike, Glen Shurig, Arran, Sco...",0.08,19.7,0.93,0.05,0.02,75.5,0.64,0.34,0.00,0.21,0.0


In [14]:
rhombox = pd.read_excel(excel_file, usecols=[0,1,13,14,15,16,17,18,19,20,21,22,23], header=1)
#could also pass a list of numbers that defines which columns to load
rhombox.head()

,Index,Label,SiO2,TiO2,Al2O3,V2O3,Cr2O3,FeO,MnO,MgO,CaO,ZnO,NiO
0,1,Hat Creek (Olivine Basalt) Table 5 (63) No: 12...,0.14,49.5,0.05,0.0,0.00,47.7,0.70,0.69,0.20,0.00,0.0
1,2,G151 Rhyolite (Thingmuli),0.05,49.8,0.04,0.0,0.00,47.7,0.98,0.62,0.06,0.10,0.0
2,3,"Oraefi (Obsidian from Oraefajokull, Eastern Ic...",0.03,49.9,0.03,0.0,0.00,47.9,1.23,0.10,0.04,0.00,0.0
3,4,"AC-5 Pitchstone (sill, Cluachland shore, Arran...",0.03,48.9,0.06,0.0,0.02,48.7,1.02,0.47,0.00,0.05,0.0
4,5,"AC-7 Pitchstone (dike, Glen Shurig, Arran, Sco...",0.04,49.6,0.06,0.0,0.00,48.3,0.82,0.63,0.02,0.00,0.0


In [15]:
#getting values from the dataframes above
SP = np.array(spinel)
RO = np.array(rhombox)

In [16]:
Mag_mol_oxides = []
Ilm_mol_oxides = []

for rows in SP:
    oxs = rows[2:] #get each row of ilmenite matrix
    Mag_mol_oxides.append(thermo.chem.format_mol_oxide_comp(
        {'SiO2':oxs[0], 'TiO2':oxs[1], 'Al2O3':oxs[2], 'Fe2O3':0.0, 'FeO':oxs[5],
       'MnO':oxs[6], 'MgO':oxs[7], 'CaO':oxs[8], 'Na2O':0.0, 'Cr2O3':oxs[4]}, convert_grams_to_moles=True))
    #leaves out V2O3, ZnO, NiO for now, and sets Fe2O3 and Na2O to 0
    
for rowi in RO:
    oxi = rowi[2:] #get each row of ilmenite matrix
    Ilm_mol_oxides.append(thermo.chem.format_mol_oxide_comp(
        {'SiO2':oxi[0], 'TiO2':oxi[1], 'Al2O3':oxi[2], 'Fe2O3':0.0, 'FeO':oxi[5],
       'MnO':oxi[6], 'MgO':oxi[7], 'CaO':oxi[8], 'Na2O':0.0, 'Cr2O3':oxi[4]}, convert_grams_to_moles=True))
    #leaves out V2O3, ZnO, NiO for now, and sets Fe2O3 and Na2O to 0

## Step 2 - Convert analytical composition to moles of endmember components
Note that a method - *test_endmember_comp()* - is called to test the validity of the projected composition  
Also note that the "intrinsic" conversion routines now take care of FeO-Fe2O3 conversion to balance the cation-anion ratio of the phase.  Input compositions of Fe2O3 and/or FeO are adjusted to balance charge and maintain phase stoichiometry.

In [17]:
def validate_endmember_comp(moles_end, phase):
    #print(phase.props['phase_name'])
    sum = 0.0
    for i in range(0,phase.props['endmember_num']):
        print("mole number of {0:10.10s} = {1:13.8f}".format(
            phase.props['endmember_name'][i], moles_end[i]))
        sum += moles_end[i]
    if not phase.test_endmember_comp(moles_end):
        print ("Calculated composition is infeasible!")

Mag_moles_end = [] #where we will store matrix
Ilm_moles_end = [] #where we will store matrix
                
for m in Mag_mol_oxides:
    Mag_moles_end.append(Mag.calc_endmember_comp(
    mol_oxide_comp=m, method='intrinsic', normalize=True))
    #validate_endmember_comp(Mag_moles_end, Mag)
    
for i in Ilm_mol_oxides: 
    Ilm_moles_end.append(Ilm.calc_endmember_comp(
    mol_oxide_comp=i, method='intrinsic', normalize=True))
    #validate_endmember_comp(Ilm_moles_end[i], Ilm)

### Calculate the equilibrium temperature for this oxide pair ...

In [18]:
P = 2000.0 #Fix the pressure at 2000 bars for now
Teq = []
for mag, ilm in zip(Mag_moles_end, Ilm_moles_end):
    Teq.append(boundary(P, [500.,2500.], deltaG, mag, ilm))

## Calculate Oxygen fugacity from the reaction 

### O2 + 4 Fe3O4 = 6 Fe2O3

Note that the properties of oxygen are defined here for consistency instead of using the built-in functions.  
Also note that the chemical potentials of hematite and magnetite are adjusted to remove the Berman-type volume integrals and replace them with the Vinet-type volume integrals to be consistent with Ghiorso and Evans (2008)

In [19]:
def muO2(t, p):
    tr = 298.15
    hs = 23.10248*(t-tr) + 2.0*804.8876*(np.sqrt(t)-np.sqrt(tr)) - 1762835.0*(1.0/t-1.0/tr) \
       - 18172.91960*np.log(t/tr) + 0.5*0.002676*(t*t-tr*tr)
    ss = 205.15 + 23.10248*np.log(t/tr)  - 2.0*804.8876*(1.0/np.sqrt(t)-1.0/np.sqrt(tr)) \
       - 0.5*1762835.0*(1.0/(t*t)-1.0/(tr*tr)) + 18172.91960*(1.0/t-1.0/tr) + 0.002676*(t-tr)
    return hs - t*ss
def deltaNNO (T, P, mag_mols, ilm_mols):
    muHem  = Ilm.chem_potential(T, P, mol=ilm_mols, endmember=1)
    muHem -= BermanVint(T, P, 3.027, -0.479e-6, 0.304e-12, 38.310e-6, 1.650e-10)
    muHem += VinetVint(T, P, 3.027, 38.310E-6, 208.768, 1.64992)
    muMag =  Mag.chem_potential(T, P, mol=mag_mols, endmember=2)
    muMag -= BermanVint(T, P, 4.452, -0.582E-6, 1.751E-12, 30.291E-6, 138.470E-10)
    muMag += VinetVint(T, P, 4.452, 30.291E-6, 171.821, 9.3387)
    muOxy  = muO2(T, P)
    logfO2 = (6.0*muHem - 4.0*muMag -  muOxy)/(8.3144598*T)/np.log(10.0)
    return logfO2 - (-25018.7/T + 12.981 + 0.046*(P-1.0)/T -0.5117*np.log(T))

### Calculate the equilibrium oxygen fugacity for this oxide pair ...

In [20]:
fO2 = []
for T, mag, ilm in zip(Teq, Mag_moles_end, Ilm_moles_end):
    fO2.append(deltaNNO(T, P, mag, ilm))

## Calculate the temperature for Fe-Mg exchange
### FeAl2O4 (hercynite) + MgTiO3 (geikielite) = MgAl2O4 (spinel) + FeTiO3 (ilmenite)
The method below is used by the minimizer to evaluate the free energy change of the Fe-Mg exchange reaction ...

In [21]:
def deltaGfemg(T, P, mag_mols, ilm_mols):
    muSpn  = Mag.chem_potential(T, P, mol=mag_mols, endmember=3)
    muSpn -= BermanVint(T, P, 3.977, -0.489E-6, 0.0, 21.691E-6, 50.528E-10)
    muSpn += VinetVint(T, P, 3.977, 21.691E-6, 204.499, 4.0)
    
    muHer  = Mag.chem_potential(T, P, mol=mag_mols, endmember=1)
    muHer -= BermanVint(T, P, 0.973948*4.184, 0.0, 0.0, 0.0, 0.0)
    muHer += VinetVint(T, P, 0.973948*4.184, 21.691E-6, 204.499, 4.0)
    
    muIlm  = Ilm.chem_potential(T, P, mol=ilm_mols, endmember=2)
    muIlm -= BermanVint(T, P, 3.170, -0.584e-6, 1.230e-12, 27.248e-6, 29.968e-10)
    muIlm += VinetVint(T, P, 3.170, 27.248E-6, 171.233, 6.21289)
    
    muGei  = Ilm.chem_potential(T, P, mol=ilm_mols, endmember=0)
    muGei -= BermanVint(T, P, 3.086, -0.584e-6, 1.230e-12, 27.248e-6, 29.968e-10)
    muGei += VinetVint(T, P, 3.086, 27.2476341e-6, 171.240, 6.21527)
    
    deltaG = muSpn + muIlm - muHer - muGei
    return deltaG

Compute the Fe-Mg exchange temperature (if possible) ...

In [22]:
TFeMg = []
for mag, ilm in zip(Mag_moles_end, Ilm_moles_end):
    Tlow  = deltaGfemg(500.0, P, mag, ilm)
    Thigh = deltaGfemg(2000.0, P, mag, ilm)
    if np.sign(Tlow) != np.sign(Thigh):
        Tfemg = boundary(P, [500.,2000.], deltaGfemg, mag, ilm)
        T_C = np.array(Tfemg) - 273.15
       # print('Fe-Mg Equilibrium Temp = ', Tfemg-273.15, ' °C')
    else:
        T_C = 'none'
       # print('No Fe-Mg equilibration temperature found.')
    TFeMg.append(T_C)

No Fe-Mg equilibration temperature found.
Fe-Mg Equilibrium Temp =  646.4557425271619  °C
Fe-Mg Equilibrium Temp =  742.453682541893  °C
Fe-Mg Equilibrium Temp =  1374.8184585629542  °C
Fe-Mg Equilibrium Temp =  940.2633567408199  °C
No Fe-Mg equilibration temperature found.
Fe-Mg Equilibrium Temp =  1144.3668075878722  °C
Fe-Mg Equilibrium Temp =  1133.3148217115777  °C
Fe-Mg Equilibrium Temp =  1011.144380579018  °C
Fe-Mg Equilibrium Temp =  1057.8691687731366  °C
Fe-Mg Equilibrium Temp =  1198.4274030374368  °C
Fe-Mg Equilibrium Temp =  696.8724625279862  °C
Fe-Mg Equilibrium Temp =  1096.1798967990026  °C
Fe-Mg Equilibrium Temp =  704.2692667976235  °C
No Fe-Mg equilibration temperature found.
Fe-Mg Equilibrium Temp =  713.5332564267131  °C
No Fe-Mg equilibration temperature found.
Fe-Mg Equilibrium Temp =  621.84316272427  °C
Fe-Mg Equilibrium Temp =  1091.749224485366  °C
Fe-Mg Equilibrium Temp =  559.6716121701219  °C
Fe-Mg Equilibrium Temp =  895.3086959025912  °C
Fe-Mg Equilib

## Calculate the activity of TiO2 relative to rutile saturation 

In [23]:
Rut = modelDB.get_phase('Rt')
def aTiO2(T, P, mag_mols, ilm_mols):
    muUlv  = Mag.chem_potential(T, P, mol=mag_mols, endmember=4) + UlvCorr(T, correctReaction=False)
    muUlv -= BermanVint(T, P, 4.682, 0.0, 0.0, 0.0, 0.0)
    muUlv += VinetVint(T, P, 4.682, 30.291E-6, 171.821, 9.3387)
    muIlm  = Ilm.chem_potential(T, P, mol=ilm_mols, endmember=2)
    muIlm -= BermanVint(T, P, 3.170, -0.584e-6, 1.230e-12, 27.248e-6, 29.968e-10)
    muIlm += VinetVint(T, P, 3.170, 27.248E-6, 171.233, 6.21289)
    muRut = Rut.chem_potential(T, P)
    return np.exp(-(muRut+muUlv-2.0*muIlm)/(8.3143*T))

In [24]:
actTiO2 = []
for T, mag, ilm in zip(Teq, Mag_moles_end, Ilm_moles_end):
    actTiO2.append(aTiO2(T, P, mag, ilm))

## Exporting the results to Excel

### Variables to output = Teq, fO2, TFeMg, actTiO2

In [25]:
data['fO2 rel NNO'] = fO2
data['T °C, Fe-Ti'] = np.array(Teq) - 273.15
data['T °C, Fe-Mg'] = TFeMg
data['TiO2 activity'] = np.array(actTiO2)

In [26]:
data.head()

,Index,Label,Spinel SiO2,Spinel TiO2,Spinel Al2O3,Spinel V2O3,Spinel Cr2O3,Spinel FeO,Spinel MnO,Spinel MgO,...,Rhomb FeO,Rhomb MnO,Rhomb MgO,Rhomb CaO,Rhomb ZnO,Rhomb NiO,fO2 rel NNO,"T °C, Fe-Ti","T °C, Fe-Mg",TiO2 activity
0,1,Hat Creek (Olivine Basalt) Table 5 (63) No: 12...,0.14,26.1,0.41,0.55,0.02,68.8,0.67,0.87,...,47.7,0.70,0.69,0.20,0.00,0.0,-1.492634,979.352298,none,0.497441
1,2,G151 Rhyolite (Thingmuli),0.13,21.5,1.15,0.00,0.00,73.6,0.88,0.32,...,47.7,0.98,0.62,0.06,0.10,0.0,-1.588021,863.611729,646.456,0.420164
2,3,"Oraefi (Obsidian from Oraefajokull, Eastern Ic...",0.11,22.1,0.64,0.00,0.01,73.0,1.00,0.06,...,47.9,1.23,0.10,0.04,0.00,0.0,-1.851335,846.588323,742.454,0.389298
3,4,"AC-5 Pitchstone (sill, Cluachland shore, Arran...",0.07,19.8,0.99,0.30,0.10,74.0,0.76,0.40,...,48.7,1.02,0.47,0.00,0.05,0.0,-1.030617,902.563333,1374.82,0.490586
4,5,"AC-7 Pitchstone (dike, Glen Shurig, Arran, Sco...",0.08,19.7,0.93,0.05,0.02,75.5,0.64,0.34,...,48.3,0.82,0.63,0.02,0.00,0.0,-1.346514,855.475008,940.263,0.437825


In [27]:
output = data
#give the filename to output to
output.to_excel('output.xlsx', index=False)
#turn off default Excel index column#give the filename that it will write to